# 06. 탐지 결과 저장 및 관리 시스템

**담당**: Claude Opus  
**작성일**: 2025-10-24  
**목적**: YOLO11 탐지 결과를 효율적으로 저장하고 관리하는 시스템

## 주요 기능
- SQLite 데이터베이스 기반 결과 저장
- JSON/CSV 파일 내보내기
- 이미지 어노테이션 저장
- 결과 검색 및 필터링
- 버전 관리 및 백업

In [ ]:
# 필요 패키지 설치
import sys
!{sys.executable} -m pip install opencv-python-headless pillow numpy sqlalchemy pandas

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import json
import sqlite3
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, Text, ForeignKey, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, Session
import pandas as pd
from pathlib import Path
from typing import List, Dict, Optional, Tuple, Any, Union
from dataclasses import dataclass, field, asdict
from datetime import datetime, timedelta
import logging
from enum import Enum
import hashlib
import shutil
import zipfile
import pickle
from collections import defaultdict
import uuid

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# SQLAlchemy Base
Base = declarative_base()

## 1. 데이터베이스 모델 정의

In [ ]:
class DetectionSession(Base):
    """탐지 세션 모델"""
    __tablename__ = 'detection_sessions'
    
    id = Column(String, primary_key=True, default=lambda: str(uuid.uuid4()))
    name = Column(String, nullable=False)
    description = Column(Text)
    created_at = Column(DateTime, default=datetime.now)
    updated_at = Column(DateTime, default=datetime.now, onupdate=datetime.now)
    model_name = Column(String)
    model_version = Column(String)
    total_images = Column(Integer, default=0)
    total_detections = Column(Integer, default=0)
    
    # 관계
    images = relationship("DetectionImage", back_populates="session", cascade="all, delete-orphan")

class DetectionImage(Base):
    """탐지 이미지 모델"""
    __tablename__ = 'detection_images'
    
    id = Column(String, primary_key=True, default=lambda: str(uuid.uuid4()))
    session_id = Column(String, ForeignKey('detection_sessions.id'))
    file_path = Column(String, nullable=False)
    file_hash = Column(String, unique=True)
    width = Column(Integer)
    height = Column(Integer)
    processed_at = Column(DateTime, default=datetime.now)
    processing_time = Column(Float)
    
    # GPS 메타데이터
    gps_latitude = Column(Float)
    gps_longitude = Column(Float)
    altitude = Column(Float)
    
    # 드론 메타데이터
    drone_model = Column(String)
    camera_model = Column(String)
    
    # 관계
    session = relationship("DetectionSession", back_populates="images")
    detections = relationship("Detection", back_populates="image", cascade="all, delete-orphan")

class Detection(Base):
    """개별 탐지 결과 모델"""
    __tablename__ = 'detections'
    
    id = Column(String, primary_key=True, default=lambda: str(uuid.uuid4()))
    image_id = Column(String, ForeignKey('detection_images.id'))
    class_name = Column(String, nullable=False)
    class_id = Column(Integer)
    confidence = Column(Float, nullable=False)
    
    # 바운딩 박스 좌표
    x1 = Column(Float, nullable=False)
    y1 = Column(Float, nullable=False)
    x2 = Column(Float, nullable=False)
    y2 = Column(Float, nullable=False)
    
    # 추가 정보
    area = Column(Float)
    center_x = Column(Float)
    center_y = Column(Float)
    
    # 관계
    image = relationship("DetectionImage", back_populates="detections")

class CropClass(Base):
    """작물 클래스 정의"""
    __tablename__ = 'crop_classes'
    
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True, nullable=False)
    korean_name = Column(String)
    description = Column(Text)
    color_rgb = Column(String)  # "255,0,0" 형식
    created_at = Column(DateTime, default=datetime.now)

## 2. 데이터베이스 매니저

In [ ]:
class DatabaseManager:
    """데이터베이스 관리 클래스"""
    
    def __init__(self, db_path: str = "detection_results.db"):
        self.db_path = db_path
        self.engine = create_engine(f'sqlite:///{db_path}', echo=False)
        Base.metadata.create_all(self.engine)
        self.SessionLocal = sessionmaker(bind=self.engine)
        
        # 기본 작물 클래스 초기화
        self._initialize_crop_classes()
        
        logger.info(f"데이터베이스 초기화: {db_path}")
    
    def _initialize_crop_classes(self):
        """기본 작물 클래스 초기화"""
        default_classes = [
            {"id": 0, "name": "wheat", "korean_name": "밀", "color_rgb": "255,215,0"},
            {"id": 1, "name": "corn", "korean_name": "옥수수", "color_rgb": "255,255,0"},
            {"id": 2, "name": "rice", "korean_name": "벼", "color_rgb": "0,255,0"},
            {"id": 3, "name": "soybean", "korean_name": "콩", "color_rgb": "139,69,19"},
            {"id": 4, "name": "potato", "korean_name": "감자", "color_rgb": "165,42,42"},
            {"id": 5, "name": "tomato", "korean_name": "토마토", "color_rgb": "255,0,0"},
            {"id": 6, "name": "pepper", "korean_name": "고추", "color_rgb": "255,69,0"},
            {"id": 7, "name": "cabbage", "korean_name": "배추", "color_rgb": "144,238,144"},
            {"id": 8, "name": "disease", "korean_name": "병해", "color_rgb": "128,0,128"},
            {"id": 9, "name": "pest", "korean_name": "충해", "color_rgb": "64,64,64"}
        ]
        
        with self.SessionLocal() as session:
            for cls in default_classes:
                existing = session.query(CropClass).filter_by(id=cls["id"]).first()
                if not existing:
                    crop_class = CropClass(**cls)
                    session.add(crop_class)
            session.commit()
    
    def create_session(self, name: str, description: str = "", 
                      model_name: str = "YOLO11", model_version: str = "1.0") -> str:
        """새 탐지 세션 생성"""
        with self.SessionLocal() as session:
            detection_session = DetectionSession(
                name=name,
                description=description,
                model_name=model_name,
                model_version=model_version
            )
            session.add(detection_session)
            session.commit()
            session_id = detection_session.id
            
        logger.info(f"새 세션 생성: {name} (ID: {session_id})")
        return session_id
    
    def get_session(self, session_id: str) -> Optional[Dict]:
        """세션 정보 조회"""
        with self.SessionLocal() as session:
            detection_session = session.query(DetectionSession).filter_by(id=session_id).first()
            if detection_session:
                return {
                    'id': detection_session.id,
                    'name': detection_session.name,
                    'description': detection_session.description,
                    'created_at': detection_session.created_at,
                    'total_images': detection_session.total_images,
                    'total_detections': detection_session.total_detections
                }
        return None
    
    def list_sessions(self) -> List[Dict]:
        """모든 세션 목록 조회"""
        with self.SessionLocal() as session:
            sessions = session.query(DetectionSession).all()
            return [{
                'id': s.id,
                'name': s.name,
                'created_at': s.created_at,
                'total_images': s.total_images,
                'total_detections': s.total_detections
            } for s in sessions]
    
    def backup_database(self, backup_dir: str = "backups"):
        """데이터베이스 백업"""
        Path(backup_dir).mkdir(exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_path = Path(backup_dir) / f"backup_{timestamp}.db"
        shutil.copy2(self.db_path, backup_path)
        logger.info(f"데이터베이스 백업 완료: {backup_path}")
        return str(backup_path)

## 3. 결과 저장 매니저

In [ ]:
@dataclass
class DetectionResult:
    """탐지 결과 데이터 클래스"""
    image_path: str
    detections: List[Dict]
    processing_time: float
    image_shape: Tuple[int, int, int]
    metadata: Dict = field(default_factory=dict)
    timestamp: datetime = field(default_factory=datetime.now)

class ResultStorageManager:
    """결과 저장 관리 클래스"""
    
    def __init__(self, db_manager: DatabaseManager, storage_dir: str = "detection_results"):
        self.db_manager = db_manager
        self.storage_dir = Path(storage_dir)
        self.storage_dir.mkdir(exist_ok=True)
        
        # 하위 디렉토리 생성
        self.images_dir = self.storage_dir / "images"
        self.annotations_dir = self.storage_dir / "annotations"
        self.exports_dir = self.storage_dir / "exports"
        
        for dir_path in [self.images_dir, self.annotations_dir, self.exports_dir]:
            dir_path.mkdir(exist_ok=True)
        
        logger.info(f"결과 저장 디렉토리: {storage_dir}")
    
    def save_detection_result(self, session_id: str, result: DetectionResult) -> str:
        """탐지 결과 저장"""
        with self.db_manager.SessionLocal() as session:
            # 세션 확인
            detection_session = session.query(DetectionSession).filter_by(id=session_id).first()
            if not detection_session:
                raise ValueError(f"세션을 찾을 수 없음: {session_id}")
            
            # 이미지 해시 계산
            file_hash = self._calculate_file_hash(result.image_path)
            
            # 중복 체크
            existing_image = session.query(DetectionImage).filter_by(file_hash=file_hash).first()
            if existing_image:
                logger.warning(f"이미지 이미 처리됨: {result.image_path}")
                return existing_image.id
            
            # 이미지 레코드 생성
            detection_image = DetectionImage(
                session_id=session_id,
                file_path=result.image_path,
                file_hash=file_hash,
                width=result.image_shape[1],
                height=result.image_shape[0],
                processing_time=result.processing_time,
                gps_latitude=result.metadata.get('gps_latitude'),
                gps_longitude=result.metadata.get('gps_longitude'),
                altitude=result.metadata.get('altitude'),
                drone_model=result.metadata.get('drone_model'),
                camera_model=result.metadata.get('camera_model')
            )
            session.add(detection_image)
            
            # 탐지 결과 저장
            for det in result.detections:
                detection = Detection(
                    image_id=detection_image.id,
                    class_name=det['class'],
                    class_id=det.get('class_id'),
                    confidence=det['confidence'],
                    x1=det['bbox'][0],
                    y1=det['bbox'][1],
                    x2=det['bbox'][2],
                    y2=det['bbox'][3],
                    area=(det['bbox'][2] - det['bbox'][0]) * (det['bbox'][3] - det['bbox'][1]),
                    center_x=(det['bbox'][0] + det['bbox'][2]) / 2,
                    center_y=(det['bbox'][1] + det['bbox'][3]) / 2
                )
                session.add(detection)
            
            # 세션 통계 업데이트
            detection_session.total_images += 1
            detection_session.total_detections += len(result.detections)
            detection_session.updated_at = datetime.now()
            
            session.commit()
            image_id = detection_image.id
            
        logger.info(f"결과 저장 완료: {result.image_path} ({len(result.detections)}개 탐지)")
        return image_id
    
    def save_annotated_image(self, image_id: str, image: np.ndarray, 
                            draw_labels: bool = True, draw_confidence: bool = True) -> str:
        """어노테이션된 이미지 저장"""
        with self.db_manager.SessionLocal() as session:
            detection_image = session.query(DetectionImage).filter_by(id=image_id).first()
            if not detection_image:
                raise ValueError(f"이미지를 찾을 수 없음: {image_id}")
            
            # 탐지 결과 가져오기
            detections = session.query(Detection).filter_by(image_id=image_id).all()
            
            # 이미지에 어노테이션 그리기
            annotated = self._draw_annotations(
                image, detections, draw_labels, draw_confidence
            )
            
            # 저장
            filename = f"annotated_{image_id}.jpg"
            save_path = self.annotations_dir / filename
            cv2.imwrite(str(save_path), cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR))
            
        logger.info(f"어노테이션 이미지 저장: {save_path}")
        return str(save_path)
    
    def _draw_annotations(self, image: np.ndarray, detections: List[Detection],
                         draw_labels: bool, draw_confidence: bool) -> np.ndarray:
        """이미지에 어노테이션 그리기"""
        annotated = image.copy()
        
        # 클래스별 색상 가져오기
        with self.db_manager.SessionLocal() as session:
            crop_classes = {c.name: c.color_rgb for c in session.query(CropClass).all()}
        
        for det in detections:
            # 색상 결정
            color_str = crop_classes.get(det.class_name, "0,255,0")
            color = tuple(map(int, color_str.split(',')))
            
            # 바운딩 박스 그리기
            x1, y1, x2, y2 = int(det.x1), int(det.y1), int(det.x2), int(det.y2)
            cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
            
            # 레이블 그리기
            if draw_labels or draw_confidence:
                label = det.class_name
                if draw_confidence:
                    label += f" {det.confidence:.2f}"
                
                # 텍스트 배경
                (text_width, text_height), _ = cv2.getTextSize(
                    label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
                )
                cv2.rectangle(
                    annotated, 
                    (x1, y1 - text_height - 4),
                    (x1 + text_width, y1),
                    color, -1
                )
                
                # 텍스트
                cv2.putText(
                    annotated, label,
                    (x1, y1 - 2),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255, 255, 255), 1
                )
        
        return annotated
    
    def _calculate_file_hash(self, file_path: str) -> str:
        """파일 해시 계산"""
        hash_md5 = hashlib.md5()
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()

## 4. 결과 조회 및 필터링

In [ ]:
class ResultQueryManager:
    """결과 조회 및 필터링 관리"""
    
    def __init__(self, db_manager: DatabaseManager):
        self.db_manager = db_manager
    
    def query_detections(self, 
                        session_id: Optional[str] = None,
                        class_name: Optional[str] = None,
                        min_confidence: float = 0.0,
                        date_from: Optional[datetime] = None,
                        date_to: Optional[datetime] = None,
                        limit: Optional[int] = None) -> pd.DataFrame:
        """탐지 결과 조회"""
        
        with self.db_manager.SessionLocal() as session:
            query = session.query(Detection).join(DetectionImage)
            
            # 필터 적용
            if session_id:
                query = query.filter(DetectionImage.session_id == session_id)
            
            if class_name:
                query = query.filter(Detection.class_name == class_name)
            
            if min_confidence > 0:
                query = query.filter(Detection.confidence >= min_confidence)
            
            if date_from:
                query = query.filter(DetectionImage.processed_at >= date_from)
            
            if date_to:
                query = query.filter(DetectionImage.processed_at <= date_to)
            
            if limit:
                query = query.limit(limit)
            
            # 결과를 DataFrame으로 변환
            results = []
            for det in query.all():
                results.append({
                    'detection_id': det.id,
                    'image_id': det.image_id,
                    'image_path': det.image.file_path,
                    'class_name': det.class_name,
                    'confidence': det.confidence,
                    'x1': det.x1,
                    'y1': det.y1,
                    'x2': det.x2,
                    'y2': det.y2,
                    'area': det.area,
                    'processed_at': det.image.processed_at,
                    'gps_latitude': det.image.gps_latitude,
                    'gps_longitude': det.image.gps_longitude
                })
            
        return pd.DataFrame(results)
    
    def get_statistics(self, session_id: str) -> Dict:
        """세션 통계 조회"""
        with self.db_manager.SessionLocal() as session:
            # 기본 통계
            detection_session = session.query(DetectionSession).filter_by(id=session_id).first()
            if not detection_session:
                return {}
            
            # 클래스별 통계
            class_stats = session.query(
                Detection.class_name,
                func.count(Detection.id).label('count'),
                func.avg(Detection.confidence).label('avg_confidence')
            ).join(DetectionImage).filter(
                DetectionImage.session_id == session_id
            ).group_by(Detection.class_name).all()
            
            # 시간별 통계
            time_stats = session.query(
                func.date(DetectionImage.processed_at).label('date'),
                func.count(DetectionImage.id).label('images'),
                func.count(Detection.id).label('detections')
            ).join(Detection).filter(
                DetectionImage.session_id == session_id
            ).group_by(func.date(DetectionImage.processed_at)).all()
            
        return {
            'session_name': detection_session.name,
            'total_images': detection_session.total_images,
            'total_detections': detection_session.total_detections,
            'class_distribution': {stat.class_name: {
                'count': stat.count,
                'avg_confidence': float(stat.avg_confidence)
            } for stat in class_stats},
            'daily_stats': [{  
                'date': stat.date.isoformat() if stat.date else None,
                'images': stat.images,
                'detections': stat.detections
            } for stat in time_stats]
        }
    
    def find_similar_detections(self, detection_id: str, threshold: float = 0.8) -> List[Dict]:
        """유사한 탐지 결과 찾기"""
        with self.db_manager.SessionLocal() as session:
            # 기준 탐지 결과
            base_detection = session.query(Detection).filter_by(id=detection_id).first()
            if not base_detection:
                return []
            
            # 같은 클래스의 탐지 결과 검색
            similar = session.query(Detection).filter(
                Detection.class_name == base_detection.class_name,
                Detection.confidence >= base_detection.confidence * threshold,
                Detection.id != detection_id
            ).limit(10).all()
            
            results = []
            for det in similar:
                # IoU 계산 (간단한 유사도)
                iou = self._calculate_iou_similarity(
                    (base_detection.x1, base_detection.y1, base_detection.x2, base_detection.y2),
                    (det.x1, det.y1, det.x2, det.y2)
                )
                
                if iou > 0.3:  # IoU 임계값
                    results.append({
                        'detection_id': det.id,
                        'image_path': det.image.file_path,
                        'confidence': det.confidence,
                        'similarity': iou
                    })
            
        return sorted(results, key=lambda x: x['similarity'], reverse=True)
    
    def _calculate_iou_similarity(self, box1: Tuple, box2: Tuple) -> float:
        """IoU 기반 유사도 계산"""
        # 면적 비율로 간단한 유사도 계산
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        
        if area1 == 0 or area2 == 0:
            return 0.0
        
        ratio = min(area1, area2) / max(area1, area2)
        return ratio

## 5. 내보내기 기능

In [ ]:
class ExportManager:
    """결과 내보내기 관리"""
    
    def __init__(self, db_manager: DatabaseManager, storage_manager: ResultStorageManager):
        self.db_manager = db_manager
        self.storage_manager = storage_manager
    
    def export_to_coco(self, session_id: str, output_path: str):
        """COCO 형식으로 내보내기"""
        with self.db_manager.SessionLocal() as session:
            # 세션 정보
            detection_session = session.query(DetectionSession).filter_by(id=session_id).first()
            if not detection_session:
                raise ValueError(f"세션을 찾을 수 없음: {session_id}")
            
            # COCO 형식 데이터 구조
            coco_data = {
                "info": {
                    "description": detection_session.description,
                    "version": "1.0",
                    "year": datetime.now().year,
                    "date_created": datetime.now().isoformat()
                },
                "images": [],
                "annotations": [],
                "categories": []
            }
            
            # 카테고리 추가
            crop_classes = session.query(CropClass).all()
            for cls in crop_classes:
                coco_data["categories"].append({
                    "id": cls.id,
                    "name": cls.name,
                    "supercategory": "crop"
                })
            
            # 이미지와 어노테이션 추가
            images = session.query(DetectionImage).filter_by(session_id=session_id).all()
            annotation_id = 1
            
            for idx, img in enumerate(images, 1):
                # 이미지 정보
                coco_data["images"].append({
                    "id": idx,
                    "file_name": Path(img.file_path).name,
                    "width": img.width,
                    "height": img.height,
                    "date_captured": img.processed_at.isoformat()
                })
                
                # 탐지 결과
                for det in img.detections:
                    width = det.x2 - det.x1
                    height = det.y2 - det.y1
                    
                    coco_data["annotations"].append({
                        "id": annotation_id,
                        "image_id": idx,
                        "category_id": det.class_id or 0,
                        "bbox": [det.x1, det.y1, width, height],
                        "area": float(det.area),
                        "iscrowd": 0,
                        "score": float(det.confidence)
                    })
                    annotation_id += 1
        
        # JSON 저장
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(coco_data, f, indent=2, ensure_ascii=False)
        
        logger.info(f"COCO 형식 내보내기 완료: {output_path}")
    
    def export_to_yolo(self, session_id: str, output_dir: str):
        """YOLO 형식으로 내보내기"""
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)
        
        labels_dir = output_path / "labels"
        labels_dir.mkdir(exist_ok=True)
        
        with self.db_manager.SessionLocal() as session:
            images = session.query(DetectionImage).filter_by(session_id=session_id).all()
            
            for img in images:
                label_file = labels_dir / f"{Path(img.file_path).stem}.txt"
                
                with open(label_file, 'w') as f:
                    for det in img.detections:
                        # YOLO 형식: class_id x_center y_center width height (정규화)
                        x_center = (det.x1 + det.x2) / 2 / img.width
                        y_center = (det.y1 + det.y2) / 2 / img.height
                        width = (det.x2 - det.x1) / img.width
                        height = (det.y2 - det.y1) / img.height
                        
                        f.write(f"{det.class_id or 0} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
        
        # 클래스 파일 생성
        classes_file = output_path / "classes.txt"
        with self.db_manager.SessionLocal() as session:
            crop_classes = session.query(CropClass).order_by(CropClass.id).all()
            with open(classes_file, 'w') as f:
                for cls in crop_classes:
                    f.write(f"{cls.name}\n")
        
        logger.info(f"YOLO 형식 내보내기 완료: {output_dir}")
    
    def export_to_csv(self, session_id: str, output_path: str):
        """CSV 형식으로 내보내기"""
        query_manager = ResultQueryManager(self.db_manager)
        df = query_manager.query_detections(session_id=session_id)
        
        df.to_csv(output_path, index=False, encoding='utf-8')
        logger.info(f"CSV 내보내기 완료: {output_path} ({len(df)}개 레코드)")
    
    def create_report(self, session_id: str, output_path: str):
        """분석 리포트 생성"""
        query_manager = ResultQueryManager(self.db_manager)
        stats = query_manager.get_statistics(session_id)
        
        report = []
        report.append("=" * 60)
        report.append(f"드론 작물 탐지 분석 리포트")
        report.append("=" * 60)
        report.append(f"\n세션: {stats.get('session_name', 'Unknown')}")
        report.append(f"생성일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append(f"\n[요약 통계]")
        report.append(f"- 총 처리 이미지: {stats.get('total_images', 0)}개")
        report.append(f"- 총 탐지 객체: {stats.get('total_detections', 0)}개")
        report.append(f"- 평균 탐지/이미지: {stats.get('total_detections', 0) / max(1, stats.get('total_images', 1)):.2f}개")
        
        report.append(f"\n[클래스별 분포]")
        class_dist = stats.get('class_distribution', {})
        for class_name, class_stats in sorted(class_dist.items(), key=lambda x: x[1]['count'], reverse=True):
            report.append(f"- {class_name}: {class_stats['count']}개 (평균 신뢰도: {class_stats['avg_confidence']:.2%})")
        
        report.append(f"\n[일별 처리 현황]")
        for day_stat in stats.get('daily_stats', []):
            if day_stat['date']:
                report.append(f"- {day_stat['date']}: {day_stat['images']}개 이미지, {day_stat['detections']}개 탐지")
        
        # 파일로 저장
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("\n".join(report))
        
        logger.info(f"리포트 생성 완료: {output_path}")
        return "\n".join(report)

## 6. 통합 사용 예제

In [ ]:
# 통합 예제: 전체 워크플로우
def demo_complete_workflow():
    """완전한 결과 저장/관리 워크플로우 데모"""
    
    print("🚀 탐지 결과 저장/관리 시스템 데모\n")
    
    # 1. 시스템 초기화
    print("[1/6] 시스템 초기화...")
    db_manager = DatabaseManager("demo_detection.db")
    storage_manager = ResultStorageManager(db_manager, "demo_results")
    query_manager = ResultQueryManager(db_manager)
    export_manager = ExportManager(db_manager, storage_manager)
    
    # 2. 새 세션 생성
    print("[2/6] 탐지 세션 생성...")
    session_id = db_manager.create_session(
        name="드론 작물 모니터링 2025-10-24",
        description="테스트 농장 작물 상태 점검",
        model_name="YOLO11",
        model_version="1.0"
    )
    print(f"  세션 ID: {session_id}")
    
    # 3. 탐지 결과 저장 (시뮬레이션)
    print("[3/6] 탐지 결과 저장...")
    
    # 가짜 탐지 결과 생성
    for i in range(5):
        result = DetectionResult(
            image_path=f"test_image_{i}.jpg",
            detections=[
                {
                    "class": np.random.choice(["wheat", "corn", "rice"]),
                    "class_id": np.random.randint(0, 3),
                    "confidence": np.random.uniform(0.7, 0.99),
                    "bbox": [np.random.randint(0, 500) for _ in range(4)]
                }
                for _ in range(np.random.randint(1, 4))
            ],
            processing_time=np.random.uniform(0.1, 0.5),
            image_shape=(640, 640, 3),
            metadata={
                "gps_latitude": 37.5 + np.random.uniform(-0.01, 0.01),
                "gps_longitude": 127.0 + np.random.uniform(-0.01, 0.01),
                "altitude": 100 + np.random.uniform(-20, 20),
                "drone_model": "DJI Mavic 3",
                "camera_model": "Hasselblad L2D-20c"
            }
        )
        
        image_id = storage_manager.save_detection_result(session_id, result)
        print(f"  이미지 {i+1}/5 저장 완료")
    
    # 4. 결과 조회
    print("\n[4/6] 결과 조회 및 통계...")
    
    # 전체 통계
    stats = query_manager.get_statistics(session_id)
    print(f"  총 이미지: {stats['total_images']}")
    print(f"  총 탐지: {stats['total_detections']}")
    print(f"  클래스 분포:")
    for class_name, class_stat in stats['class_distribution'].items():
        print(f"    - {class_name}: {class_stat['count']}개")
    
    # 5. 내보내기
    print("\n[5/6] 결과 내보내기...")
    
    # CSV 내보내기
    export_manager.export_to_csv(session_id, "demo_results/export.csv")
    print("  ✓ CSV 내보내기 완료")
    
    # 리포트 생성
    report = export_manager.create_report(session_id, "demo_results/report.txt")
    print("  ✓ 리포트 생성 완료")
    
    # 6. 백업
    print("\n[6/6] 데이터베이스 백업...")
    backup_path = db_manager.backup_database("demo_results/backups")
    print(f"  ✓ 백업 완료: {backup_path}")
    
    print("\n✅ 전체 워크플로우 완료!")
    
    return session_id

# 데모 실행
demo_session_id = demo_complete_workflow()

## 7. 고급 기능: 지리공간 분석

In [ ]:
class GeoSpatialAnalyzer:
    """GPS 기반 지리공간 분석"""
    
    def __init__(self, db_manager: DatabaseManager):
        self.db_manager = db_manager
    
    def get_detection_heatmap_data(self, session_id: str, class_name: Optional[str] = None) -> List[Dict]:
        """탐지 히트맵 데이터 생성"""
        with self.db_manager.SessionLocal() as session:
            query = session.query(
                DetectionImage.gps_latitude,
                DetectionImage.gps_longitude,
                func.count(Detection.id).label('detection_count')
            ).join(Detection).filter(
                DetectionImage.session_id == session_id,
                DetectionImage.gps_latitude.isnot(None),
                DetectionImage.gps_longitude.isnot(None)
            )
            
            if class_name:
                query = query.filter(Detection.class_name == class_name)
            
            results = query.group_by(
                DetectionImage.gps_latitude,
                DetectionImage.gps_longitude
            ).all()
            
        return [{
            'lat': float(r.gps_latitude),
            'lon': float(r.gps_longitude),
            'count': r.detection_count
        } for r in results]
    
    def find_clusters(self, session_id: str, class_name: str, radius_meters: float = 50) -> List[Dict]:
        """특정 클래스의 군집 찾기"""
        # 간단한 거리 기반 클러스터링
        data = self.get_detection_heatmap_data(session_id, class_name)
        
        clusters = []
        visited = set()
        
        for i, point in enumerate(data):
            if i in visited:
                continue
            
            cluster = [point]
            visited.add(i)
            
            for j, other in enumerate(data):
                if j in visited:
                    continue
                
                # 하버사인 거리 계산 (간략화)
                distance = self._haversine_distance(
                    point['lat'], point['lon'],
                    other['lat'], other['lon']
                )
                
                if distance <= radius_meters:
                    cluster.append(other)
                    visited.add(j)
            
            if len(cluster) > 1:
                clusters.append({
                    'center_lat': np.mean([p['lat'] for p in cluster]),
                    'center_lon': np.mean([p['lon'] for p in cluster]),
                    'size': len(cluster),
                    'total_detections': sum(p['count'] for p in cluster)
                })
        
        return sorted(clusters, key=lambda x: x['total_detections'], reverse=True)
    
    def _haversine_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
        """두 GPS 좌표 간 거리 계산 (미터)"""
        from math import radians, sin, cos, sqrt, atan2
        
        R = 6371000  # 지구 반경 (미터)
        
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        
        return R * c

# 지리공간 분석 예제
print("\n🗺️ 지리공간 분석 기능")

if 'demo_session_id' in locals():
    geo_analyzer = GeoSpatialAnalyzer(db_manager)
    
    # 히트맵 데이터
    heatmap_data = geo_analyzer.get_detection_heatmap_data(demo_session_id)
    print(f"\n히트맵 데이터 포인트: {len(heatmap_data)}개")
    
    # 클러스터 찾기
    # clusters = geo_analyzer.find_clusters(demo_session_id, "wheat", radius_meters=100)
    # print(f"발견된 클러스터: {len(clusters)}개")

## 8. 성능 최적화 팁

In [ ]:
print("\n📊 성능 최적화 가이드\n")

print("1. 데이터베이스 최적화:")
print("   - 인덱스 생성으로 조회 속도 향상")
print("   - 정기적 VACUUM 실행")
print("   - 배치 삽입 사용")

print("\n2. 스토리지 최적화:")
print("   - 이미지 압축 사용")
print("   - 썸네일 생성 및 캐싱")
print("   - 오래된 데이터 아카이빙")

print("\n3. 메모리 최적화:")
print("   - 대용량 데이터는 스트리밍 처리")
print("   - 쿼리 결과 페이징")
print("   - 세션 풀 사용")

print("\n✅ 결과 저장 및 관리 시스템 구현 완료!")

## 9. 다음 모듈 연동

In [ ]:
print("\n🔗 다음 모듈 연동 정보\n")

print("결과 저장/관리 시스템이 완성되었습니다!\n")

print("이 모듈은 다음과 같이 연동됩니다:\n")

print("1. ← Todo 5 (배치 처리):")
print("   - BatchResult를 DetectionResult로 변환")
print("   - 세션별로 결과 저장")

print("\n2. → Todo 7 (스케줄링):")
print("   - 정기적 백업 스케줄링")
print("   - 오래된 데이터 정리")

print("\n3. → Todo 8 (시각화):")
print("   - 저장된 데이터 기반 차트 생성")
print("   - 지리공간 시각화")

print("\n주요 특징:")
print("✓ SQLite 데이터베이스 기반")
print("✓ 다양한 형식 내보내기")
print("✓ GPS 기반 지리공간 분석")
print("✓ 버전 관리 및 백업")
print("✓ 효율적 쿼리 및 필터링")

print("\n탐지 결과 저장/관리 시스템 개발 완료! ✨")